### Installations

In [ ]:
!pip install openai

In [ ]:
!pip install chromadb

In [ ]:
!pip install llama_index

In [ ]:
!pip install llama-index-core

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install python-dotenv

In [ ]:
%pip install llama-index-vector-stores-chroma

In [ ]:
%pip install llama-index-embeddings-langchain

In [ ]:
!pip install llama-index-llms-openai

In [ ]:
%pip install llama-index-llms-openai

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_core

###Imports

In [ ]:
import pandas as pd
import sys
import argparse
import os
import json
import pickle
import glob
import time
from datetime import datetime
import openai

In [ ]:
import chromadb
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import Prompt
from llama_index.llms.openai import OpenAI
from langchain_core.prompts import PromptTemplate

In [ ]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


###Functions

In [ ]:
args = argparse.Namespace(config_path="/content/drive/My Drive/DS340/config.json")

In [ ]:
with open(args.config_path) as json_file:
  config_dict = json.load(json_file)

#with open(args2.questions_path) as json_file:
  #questions_dict = json.load(json_file)

In [ ]:
CHUNK_SIZE = 512
CHUNK_OVERLAP = 32

In [ ]:
# Works
def initialize_and_return_models(config_dict):
    os.environ["OPENAI_API_KEY"] = config_dict['openai_api_key']
    load_dotenv("openai.env")
    openai.api_key=os.getenv('OPENAI_API_KEY')
    llm = OpenAI(model='gpt-3.5-turbo', temperature=0.5)
    embedding_model = LangchainEmbedding(
        HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    )
    return llm, embedding_model

In [ ]:
# Works
def load_target_dfs(config_dict):
    with open(config_dict['targets_train_df_path'], 'rb') as handle:
        df_train = pickle.load(handle)
    with open(config_dict['targets_test_df_path'], 'rb') as handle:
        df_test = pickle.load(handle)
    #Convert report_date column to string representation
    df_train['report_date'] = df_train['report_date'].apply(lambda x: x.date().strftime('%Y-%m-%d'))
    df_train.reset_index(drop=True, inplace=True)
    df_test['report_date'] = df_test['report_date'].apply(lambda x: x.date().strftime('%Y-%m-%d'))
    df_test.reset_index(drop=True, inplace=True)
    return df_train, df_test

In [ ]:
# Works
def get_systemprompt_template(config_dict):
    chat_text_qa_msgs = [
        SystemMessagePromptTemplate.from_template(
            config_dict['llm_system_prompt']
        ),
        HumanMessagePromptTemplate.from_template(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information, "
            "answer the question: {query_str}\n"
        ),
    ]
    chat_text_qa_msgs_lc = ChatPromptTemplate.from_messages(chat_text_qa_msgs)
    #text_qa_template = Prompt.from_langchain_prompt(chat_text_qa_msgs_lc)
    #text_qa_template = PromptTemplate.from_template(str(chat_text_qa_msgs_lc))
    text_qa_template = Prompt(str(chat_text_qa_msgs_lc))
    return text_qa_template

In [ ]:
get_systemprompt_template(config_dict)

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="input_variables=['context_str', 'query_str'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a highly experienced financial analyst. Respond to the question presented to you by assigning a confidence score ranging from 1 to 100 based on the available information. Provide your response in JSON format with only one key called score.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context_str', 'query_str'], template='Context information is below.\\n---------------------\\n{context_str}\\n---------------------\\nGiven the context information, answer the question: {query_str}\\n'))]")

In [ ]:
# Requires Updated API
def get_gpt_generated_feature_dict(query_engine, questions_dict):
    '''
    Returns:
        A dictionary with keys as question identifiers and value as GPT scores.
    '''
    response_dict = {}
    for feature_name, question in questions_dict.items():
        #Sleep for a short duration, not to exceed openai rate limits.
        time.sleep(0.2)
        response = query_engine.query(question)
        try:
          response_dict[feature_name] = int(eval(response.response)['score'])
        except:
          continue
    return response_dict

In [ ]:
def load_index(llm, embedding_model, base_embeddings_path, symbol, ar_date):
    '''
    Function to load the embeddings that were saved using embeddings_save.py
    '''
    print(os.path.join(base_embeddings_path, symbol, ar_date))
    db = chromadb.PersistentClient(path=os.path.join(base_embeddings_path, symbol, ar_date))
    chroma_collection = db.get_collection("ar_date")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    service_context = ServiceContext.from_defaults(embed_model=embedding_model,
                                                   llm=llm,
                                                   chunk_size = CHUNK_SIZE,
                                                   chunk_overlap=CHUNK_OVERLAP)
    index = VectorStoreIndex.from_vector_store(
                vector_store,
                service_context=service_context,
            )
    return index

In [ ]:
# Works
def load_index2(llm, embedding_model, base_embeddings_path, symbol, ar_date):
    '''
    Function to load the embeddings that were saved using embeddings_save.py
    '''
    try:
        db = chromadb.PersistentClient(path=os.path.join(base_embeddings_path, symbol, ar_date))
        chroma_collection = db.get_collection("ar_date")
    except Exception as e:
        print("Error:", e)
        #return "DNE"  # Return "DNE" if there is an error

    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    service_context = ServiceContext.from_defaults(embed_model=embedding_model,
                                                   llm=llm,
                                                   chunk_size = CHUNK_SIZE,
                                                   chunk_overlap=CHUNK_OVERLAP)
    index = VectorStoreIndex.from_vector_store(
                vector_store,
                service_context=service_context,
            )
    return index

In [ ]:
# Works
def load_query_engine(index, text_qa_template):
    return index.as_query_engine(text_qa_template=text_qa_template)

In [ ]:
# Works
def are_features_generated(base_path, symbol, ar_date):
    '''
    Function to check if the features df has already been created before.
    '''
    df_name = 'df_{}_{}.pickle'.format(symbol, ar_date)
    full_path = os.path.join(base_path, df_name)
    if os.path.exists(full_path):
        return True
    return False

In [ ]:
def save_features(df, llm, embedding_model, config_dict, questions_dict,
                  embeddings_directory, features_save_directory):
    '''
    Function to iteratively save features as a df with single row.
    '''
    for i in df.index:
        start_time = time.time()
        curr_series = df.loc[i]
        symbol = curr_series['symbol']
        ar_date = curr_series['report_date']
        if are_features_generated(features_save_directory, symbol, ar_date):
            continue
        index = load_index(llm, embedding_model, embeddings_directory, symbol, ar_date)
        text_qa_template = get_systemprompt_template(config_dict)
        query_engine = load_query_engine(index, text_qa_template)
        #Get feature scores as dictionary
        gpt_feature_dict = get_gpt_generated_feature_dict(query_engine, questions_dict)
        #Convert dictionary to dataframe
        gpt_feature_df = pd.DataFrame.from_dict(gpt_feature_dict, orient='index').T
        gpt_feature_df.columns = ['feature_{}'.format(c) for c in gpt_feature_df.columns]
        gpt_feature_df['meta_symbol'] = symbol
        gpt_feature_df['meta_report_date'] = ar_date
        with open(os.path.join(features_save_directory, 'df_{}_{}.pickle'.format(symbol, ar_date)), 'wb') as handle:
            pickle.dump(gpt_feature_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Completed: {} in {:.2f}s".format(i, time.time()-start_time))

In [ ]:
def save_consolidated_df(config_dict, questions_dict, targets_df,
                         features_save_directory, final_df_save_path):
    df_paths_list = [file for file in glob.glob(os.path.join(features_save_directory, '*')) \
                  if os.path.isfile(file)]
    feature_df_full = pd.DataFrame()
    feature_cols = list(questions_dict.keys())
    feature_cols = ['feature_{}'.format(f) for f in feature_cols]
    meta_cols = ['meta_symbol', 'meta_report_date']
    for df_path in df_paths_list:
        with open(df_path, 'rb') as handle:
            gpt_feature_df = pickle.load(handle)
        gpt_feature_df = gpt_feature_df.loc[:, feature_cols + meta_cols].copy()
        feature_df_full = pd.concat([feature_df_full, gpt_feature_df], ignore_index=True)
    #Convert meta_report_date column to datetime format
    feature_df_full['meta_report_date'] = feature_df_full['meta_report_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    merged_df = pd.merge(feature_df_full, targets_df, left_on=['meta_symbol', 'meta_report_date'],
                        right_on=['symbol', 'report_date'], how='inner')
    #Transform features in range [0,1]
    merged_df[feature_cols] = merged_df[feature_cols]/100.0
    with open(final_df_save_path, 'wb') as handle:
        pickle.dump(merged_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
'''
def main(args):
    with open(args.config_path) as json_file:
        config_dict = json.load(json_file)
    with open(args.questions_path) as json_file:
        questions_dict = json.load(json_file)

    df_train, df_test = load_target_dfs(config_dict)
    llm, embedding_model = initialize_and_return_models(config_dict)

    save_features(df_train, llm, embedding_model, config_dict, questions_dict,
                  embeddings_directory=config_dict['embeddings_for_training_directory'],
                  features_save_directory=config_dict['feature_train_df_save_directory'])
    save_features(df_test, llm, embedding_model, config_dict, questions_dict,
                  embeddings_directory=config_dict['embeddings_for_testing_directory'],
                  features_save_directory=config_dict['feature_test_df_save_directory'])

    save_consolidated_df(config_dict, questions_dict, df_train,
                         features_save_directory=config_dict['feature_train_df_save_directory'],
                         final_df_save_path=config_dict['final_train_df_save_path'])
    save_consolidated_df(config_dict, questions_dict, df_test,
                         features_save_directory=config_dict['feature_test_df_save_directory'],
                         final_df_save_path=config_dict['final_test_df_save_path'])
'''

In [ ]:
args = argparse.Namespace(config_path="/content/drive/My Drive/DS340/config.json")

In [ ]:
args2 = argparse.Namespace(questions_path="/content/drive/My Drive/DS340/questions_dict.json")

### Modified Implementation

In [ ]:
with open(args.config_path) as json_file:
  config_dict = json.load(json_file)

with open(args2.questions_path) as json_file:
  questions_dict = json.load(json_file)

In [ ]:
train_path = "/content/drive/My Drive/DS340/df_train.csv"
test_path = "/content/drive/My Drive/DS340/df_test.csv"

In [ ]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [ ]:
df_train.shape

(392, 24)

In [ ]:
df_train.columns

Index(['target_max', 'target_min', 'target_3m', 'target_6m', 'target_9m',
       'target_12m', 'sp500_max', 'sp500_min', 'sp500_3m', 'sp500_6m',
       'sp500_9m', 'sp500_12m', 'report_date', 'start_date', 'end_date',
       'symbol', 'era', 'target_max_normalised', 'target_min_normalised',
       'target_3m_normalised', 'target_6m_normalised', 'target_9m_normalised',
       'target_12m_normalised', 'target_ml'],
      dtype='object')

In [ ]:
llm, embedding_model = initialize_and_return_models(config_dict)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
symbol = "MAT"
ar_date = "2012-02-23"

In [ ]:
index= load_index(llm, embedding_model, config_dict['embeddings_for_training_directory'], symbol, ar_date)

In [ ]:
# ACIW was moved from previous embeddings, this shows that previous embeddings are not corrupted. Chromasplite file appears after running this
index= load_index2(llm, embedding_model, config_dict['embeddings_for_training_directory'], symbol, ar_date)

<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


In [ ]:
text_qa_template = get_systemprompt_template(config_dict)
query_engine = load_query_engine(index, text_qa_template)

In [ ]:
gpt_feature_dict = get_gpt_generated_feature_dict(query_engine, questions_dict)

In [ ]:
def save_features2(df, llm, embedding_model, config_dict, questions_dict,
                               embeddings_directory, features_save_directory,
                               start_index=0, end_index=None):
    '''
    Function to iteratively save features as a df with single row.
    '''
    num_rows = len(df)

    if end_index is None:
        end_index = num_rows

    for i in range(start_index, min(end_index, num_rows)):
        start_time = time.time()
        curr_series = df.loc[i]
        symbol = curr_series['symbol']
        ar_date = curr_series['report_date']
        if are_features_generated(features_save_directory, symbol, ar_date):
            continue
        try:
          index = load_index2(llm, embedding_model, embeddings_directory, symbol, ar_date)
        except:
          continue
        text_qa_template = get_systemprompt_template(config_dict)
        try:
          query_engine = load_query_engine(index, text_qa_template)
        except:
          continue
        # Get feature scores as dictionary
        gpt_feature_dict = get_gpt_generated_feature_dict(query_engine, questions_dict)
        # Convert dictionary to dataframe
        gpt_feature_df = pd.DataFrame.from_dict(gpt_feature_dict, orient='index').T
        gpt_feature_df.columns = ['feature_{}'.format(c) for c in gpt_feature_df.columns]
        gpt_feature_df['meta_symbol'] = symbol
        gpt_feature_df['meta_report_date'] = ar_date
        with open(os.path.join(features_save_directory, 'df_{}_{}.pickle'.format(symbol, ar_date)), 'wb') as handle:
          pickle.dump(gpt_feature_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Completed: {} in {:.2f}s".format(i, time.time()-start_time))


In [ ]:
# 392
def save_features_train(start_index, end_index):
  save_features2(df_train, llm, embedding_model, config_dict, questions_dict, embeddings_directory=config_dict['embeddings_for_training_directory'],
                 features_save_directory=config_dict['feature_train_df_save_directory'], start_index = start_index, end_index = end_index)

In [ ]:
# 201
def save_features_test(start_index, end_index):
  save_features2(df_test, llm, embedding_model, config_dict, questions_dict, embeddings_directory=config_dict['embeddings_for_testing_directory'],
                 features_save_directory=config_dict['feature_test_df_save_directory'], start_index = start_index, end_index = end_index)

### Embedding

In [ ]:
save_features_train(1,10)

<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 1 in 50.13s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 2 in 10.29s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 7 in 8.73s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 8 in 12.13s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


In [ ]:
save_features_train(11,21)

<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 11 in 13.24s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 12 in 19.51s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 18 in 12.75s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 19 in 13.43s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 20 in 13.40s


In [ ]:
save_features_train(21,41)

<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 21 in 9.77s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 22 in 10.94s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 31 in 12.34s


<ipython-input-63-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 32 in 9.87s
Error: Collection ar_date does not exist.


AttributeError: 'str' object has no attribute 'as_query_engine'

In [ ]:
save_features_train(33,41)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(41,51)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


<ipython-input-112-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 47 in 10.66s


<ipython-input-112-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 48 in 18.55s
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(71,91)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


<ipython-input-112-2a877f65ef0d>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 86 in 11.40s
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(102,200)

### Chroma availblity testing

In [ ]:
with open(args.config_path) as json_file:
  config_dict = json.load(json_file)

In [ ]:
config_dict['feature_test_df_save_directory']

'/content/drive/My Drive/DS340/test_features2'

In [ ]:
train_path = "/content/drive/My Drive/DS340/new_df_train.csv"
test_path = "/content/drive/My Drive/DS340/new_df_test.csv"

In [ ]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [ ]:
df_test.shape

(201, 24)

In [ ]:
def save_features_train(start_index, end_index):
  save_features2(df_train, llm, embedding_model, config_dict, questions_dict, embeddings_directory=config_dict['embeddings_for_training_directory'],
                 features_save_directory=config_dict['feature_train_df_save_directory'], start_index = start_index, end_index = end_index)

In [ ]:
def save_features_test(start_index, end_index):
  save_features2(df_test, llm, embedding_model, config_dict, questions_dict, embeddings_directory=config_dict['embeddings_for_testing_directory'],
                 features_save_directory=config_dict['feature_test_df_save_directory'], start_index = start_index, end_index = end_index)

In [ ]:
save_features_train(0,10)

<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 0 in 137.96s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 1 in 132.74s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 7 in 74.21s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 8 in 112.11s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 9 in 131.96s


In [ ]:
save_features_train(10,20)

<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 10 in 101.04s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 11 in 130.73s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 17 in 118.36s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 18 in 131.54s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 19 in 112.00s


In [ ]:
save_features_train(20,40)

<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 20 in 110.56s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 21 in 128.12s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 30 in 115.93s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 31 in 101.31s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 32 in 122.46s
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(35,45)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(45,66)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 47 in 75.98s


<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 48 in 150.10s
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(66,86)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(86,114)

<ipython-input-55-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 86 in 117.64s
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Co

In [ ]:
save_features_train(114,134)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


In [ ]:
save_features_train(134,164)

Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.
Error: Collection ar_date does not exist.


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 139 in 117.50s
Error: Collection ar_date does not exist.


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 141 in 220.66s


...

Completed: 160 in 114.03s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 161 in 93.81s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 162 in 185.67s


In [ ]:
df_train.shape

(163, 24)

In [ ]:
save_features_test(0,10)

<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 0 in 99.28s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 1 in 151.43s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 7 in 101.67s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 8 in 162.66s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 9 in 109.04s


In [ ]:
save_features_test(10,20)

<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 10 in 100.91s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 11 in 97.41s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 17 in 97.06s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 18 in 150.08s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 19 in 144.73s


In [ ]:
save_features_test(20,30)

<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 20 in 109.05s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 21 in 105.40s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 27 in 114.29s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 28 in 123.04s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 29 in 259.31s


In [ ]:
save_features_test(30,40)

<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 30 in 113.61s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 31 in 119.45s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


...

Completed: 37 in 109.56s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 38 in 123.36s


<ipython-input-31-ec74b98654fd>:14: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model,


Completed: 39 in 151.76s


### Save Final DataFrames

In [ ]:
#save_consolidated_df(config_dict, questions_dict, df_train, features_save_directory=config_dict['feature_train_df_save_directory'],final_df_save_path=config_dict['final_train_df_save_path'])

KeyError: "['feature_management'] not in index"

In [ ]:
#save_consolidated_df(config_dict, questions_dict, df_test, features_save_directory=config_dict['feature_test_df_save_directory'], final_df_save_path=config_dict['final_test_df_save_path'])